In [ ]:
!pip install surprise

import pandas as pd
import json
import gzip
import numpy as np
import random
from surprise import accuracy
from surprise import Dataset
from surprise import Reader
from surprise import SVD,SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering

from surprise.model_selection import GridSearchCV
from tqdm import tqdm
import matplotlib.pyplot as plt
from surprise.model_selection.validation import cross_validate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163346 sha256=4d49362d6a71f5aeb5f3df2d2a25e21397510b92d6e52f560e1ecea0e4756b8c
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [1]:
!python --version

Python 3.10.12


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Input, Embedding, Concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import keras
from keras.layers import Add, Activation, Lambda, BatchNormalization, Concatenate, Dropout, Input, Embedding, Dot, Reshape, Dense, Flatten
from keras import regularizers
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

!pip install LibRecommender
from libreco.data import random_split, DatasetPure
from libreco.algorithms import NCF
from libreco.evaluation import evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.0 MB/s eta 0:00:00


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

news_df = pd.read_csv('/content/drive/MyDrive/295A/MINDlarge_train/news.tsv', sep='\t', header=None, names=['NewsID', 'Category', 'Subcategory', 'Title', 'Abstract', 'URL', 'Title Entities', 'Abstract Entities'])
behaviors_df = pd.read_csv('/content/drive/MyDrive/295A/MINDlarge_train/behaviors.tsv', sep='\t', header=None, names=['ImpressionID', 'UserID', 'Time', 'History', 'Impressions'])

Mounted at /content/drive


In [ ]:
news_df = news_df[['NewsID']]
behaviors_df = behaviors_df[['UserID', 'Impressions']]

behaviors_df.head()

,UserID,Impressions
0,U87243,N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...
1,U598644,N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...
2,U532401,N103852-0 N53474-0 N127836-0 N47925-1
3,U593596,N38902-0 N76434-0 N71593-0 N100073-0 N108736-0...
4,U239687,N76209-0 N48841-0 N67937-0 N62235-0 N6307-0 N3...


In [ ]:
behaviors_df.shape

(2232748, 2)

In [ ]:
mask = behaviors_df['Impressions'].str.contains('-1') & behaviors_df['Impressions'].str.contains('-0')
balanced_behaviors = behaviors_df[mask]

In [ ]:
# Sample 15% from balanced behaviors
sampled_behaviors = balanced_behaviors.sample(frac=0.15)
print(sampled_behaviors.shape)

# Extract interactions from sampled behaviors
interactions = []
for _, row in sampled_behaviors.iterrows():
    for impression in row['Impressions'].split(' '):
        news_id, interaction = impression.split('-')
        interactions.append([row['UserID'], news_id, int(interaction)])

interactions_df = pd.DataFrame(interactions, columns=['UserID', 'NewsID', 'Rating'])

(334912, 2)


In [ ]:
interactions_df["user"] = interactions_df["UserID"]
interactions_df["label"] = interactions_df["Rating"]
#, "item", "label", "time"
interactions_df["item"] = interactions_df["NewsID"]
interactions_df = interactions_df[["user", "item", "label"]]

In [ ]:
train_data, eval_data, test_data = random_split(interactions_df, multi_ratios=[0.8, 0.1, 0.1])

In [ ]:
train_data, data_info= DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
test_data = DatasetPure.build_testset(test_data)

In [ ]:
ncf = NCF(
    task="rating",
    data_info=data_info,
    loss_type="cross_entropy",
    embed_size=16,
    n_epochs=5,
    lr=1e-3,
    batch_size=2048,
    num_neg=1,
)

In [ ]:
ncf.fit(
    train_data,
    neg_sampling=False,
    verbose=2,
    eval_data=eval_data,
    metrics=["loss"],
)


/usr/local/lib/python3.10/dist-packages/libreco/layers/dense.py:31: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
Instructions for updating:
Colocations handled automatically by placer.


Training start time: 2023-10-30 03:11:38


/usr/local/lib/python3.10/dist-packages/libreco/layers/dense.py:39: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
train: 100%|██████████| 9800/9800 [01:03<00:00, 153.16it/s]


Epoch 1 elapsed: 63.991s
	 train_loss: 0.0385


eval_pointwise: 100%|██████████| 154/154 [00:00<00:00, 345.85it/s]


...

	 eval rmse: 0.2004


train: 100%|██████████| 9800/9800 [00:57<00:00, 169.12it/s]


Epoch 5 elapsed: 57.956s
	 train_loss: 0.0253


eval_pointwise: 100%|██████████| 154/154 [00:00<00:00, 381.38it/s]


	 eval rmse: 0.2061


In [ ]:
evaluate(
    model=ncf,
    data=test_data,
    neg_sampling=False,
    metrics=["loss"],
)

eval_pointwise: 100%|██████████| 153/153 [00:00<00:00, 316.82it/s]


{'loss': 0.20582989}

In [ ]:
evaluate(
    model=ncf,
    data=test_data,
    neg_sampling=False,
    metrics=["mae"],
)

eval_pointwise: 100%|██████████| 153/153 [00:00<00:00, 346.23it/s]


{'mae': 0.07468366}

In [ ]:
evaluate(
    model=ncf,
    data=test_data,
    neg_sampling=False,
    metrics=["rmse"],
)

eval_pointwise: 100%|██████████| 153/153 [00:00<00:00, 236.84it/s]


{'rmse': 0.20582989}